<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/Build_groundTruth__deduping_Title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
# as of today
import datetime as dt
dt.date.today()
import os
import pandas as pd
from google.colab import drive
import csv
import os


Mounted at /content/drive


In [8]:
df_econ = pd.read_csv('/content/drive/MyDrive/finalMetaFile/metadata_econlit.csv')
df_econ['title'] = df_econ['title'].str.replace('"', "'")
df_econ=df_econ[['title']]
df_econ.head()


,title
0,Microcosms and Macrocosms: Capitalism and the Latent Metaphysics of Scale: Foreword
1,Stochastic Radiotherapy Appointment Scheduling
2,Examining the Balance between Efficiency and Resilience in Closed-Loop Supply Chains
3,The Variable-Width Strip Packing Problem
4,Constrained Naive Bayes with Application to Unbalanced Data Classification


In [3]:
df_3ie = pd.read_csv('/content/drive/MyDrive/finalMetaFile/3ie_parsed_metadata.csv')
df_3ie['title'] = df_3ie['title'].str.replace('"', "'")
df_3ie=df_3ie[['title']]
df_3ie.head()

,title
0,impacts of payments for ecosystem services pro...
1,providing collateral and improving product mar...
2,seeing is believing? evidence from an extensio...
3,smallholder access to weather securities in in...
4,sustainability of impact dimensions of decline...


# Build a ground_truth DF

## But before that, how many articles re-appear within a same source? (duplicate values within each sources)

In [4]:
df = df_3ie

# Find duplicate strings in the column
duplicates = df['title'].duplicated()

# Filter the original DataFrame to get the duplicates
duplicate_values = df[duplicates]

# Print the duplicate values
print("Duplicate Values:")
print(duplicate_values)

Duplicate Values:
                                                   title
243    heterogeneous impact of a collective payment f...
282    market access, organic farming and productivit...
287    the sloping land conversion program in china: ...
639    the effect of monitoring: how data collection ...
660    impact of fadama-ii project on poverty reducti...
...                                                  ...
10511  can public works increase equilibrium wages? e...
10575  girl empower – a gender transformative mentori...
11143  effectiveness of nutrition and wash/malaria ed...
11214  positive impact of a cooking skills interventi...
11219  effects of the rango cards game intervention o...

[132 rows x 1 columns]


In [5]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


<ipython-input-5-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [6]:
print(duplicate_values)

                                                                                                                                                                                                                                                       title
243    heterogeneous impact of a collective payment for environmental services scheme on reducing deforestation in cambodia                                                                                                                                 
282    market access, organic farming and productivity: the effects of fair trade affiliation on thai farmer producers groups                                                                                                                               
287    the sloping land conversion program in china: effect on rural households' livelihood diversification                                                                                                                                      

In [7]:

index_num = df_3ie[df_3ie['title'] == "the impact of progresa on food consumption"].index.tolist()
print(index_num)


[822, 8693]


In [16]:
import collections
counts = collections.Counter(duplicate_values['title'])
duplicate_values['count'] = duplicate_values['title'].apply(lambda x: counts[x])
print(duplicate_values)


                                                                                                                                                                                                                                                       title  \
243    heterogeneous impact of a collective payment for environmental services scheme on reducing deforestation in cambodia                                                                                                                                    
282    market access, organic farming and productivity: the effects of fair trade affiliation on thai farmer producers groups                                                                                                                                  
287    the sloping land conversion program in china: effect on rural households' livelihood diversification                                                                                                                             

<ipython-input-16-22aa145a4605>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicate_values['count'] = duplicate_values['title'].apply(lambda x: counts[x])


so there are cases when same article appears multiple times within a source.

In [9]:

def find_duplicate_titles(df):
  """
  This function takes a DataFrame as input and returns a DataFrame containing duplicate titles.
  Args: df: A Pandas DataFrame.
  Returns:  A Pandas DataFrame containing duplicate titles.
  """
  duplicates = df['title'].duplicated()
  duplicate_values = df[duplicates]
  return duplicate_values




In [11]:
directory = '/content/drive/MyDrive/finalMetaFile'
final_file_list=['metadata_jstor.csv',
                 'metadata_scopus 2.csv',
                  'elsevier_metadata.csv',
                 'metadata_semantic_scholar.csv',
                 'metadata_econlit.csv',
                 'SSRN_metadata.csv',
                 'log.csv','3ie_parsed_metadata'
                 'metadata_repec.csv']

dataframes = []

for filename in final_file_list:
  if os.path.isfile(os.path.join(directory, filename)) and filename.endswith(".csv"):
    df = pd.read_csv(os.path.join(directory, filename))
    if "title" in df.columns:
      df['title'] = df['title'].str.replace('"', "'")
      df=df[['title']]
      duplicate_titles = find_duplicate_titles(df)
      print("Number of Duplicate Titles at",filename,":",len(duplicate_titles),"(",len(duplicate_titles)/len(df),"%)")

<ipython-input-11-aa97a3aa10e9>:15: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename))


Number of Duplicate Titles at metadata_jstor.csv : 69595 ( 0.06353398490776868 %)
Number of Duplicate Titles at metadata_scopus 2.csv : 1848 ( 0.0025962932575050858 %)
Number of Duplicate Titles at elsevier_metadata.csv : 1896 ( 0.007821459510746257 %)


<ipython-input-11-aa97a3aa10e9>:15: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename))


Number of Duplicate Titles at metadata_semantic_scholar.csv : 188077 ( 0.023769974560110065 %)
Number of Duplicate Titles at metadata_econlit.csv : 216 ( 0.011296480309607239 %)


<ipython-input-11-aa97a3aa10e9>:15: DtypeWarning: Columns (4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename))


Number of Duplicate Titles at metadata_repec.csv : 78883 ( 0.08613834681377767 %)


In [12]:
final_file_list=['metadata_jstor.csv',
                 'metadata_scopus 2.csv',
                  'elsevier_metadata.csv',
                 'metadata_semantic_scholar.csv',
                 'metadata_econlit.csv',
                 'SSRN_metadata.csv',
                 'log.csv','3ie_parsed_metadata'
                 'metadata_repec.csv']

total_duplicates = 0
total_titles=0
for filename in final_file_list:
  if os.path.isfile(os.path.join(directory, filename)) and filename.endswith(".csv"):
    df = pd.read_csv(os.path.join(directory, filename))
    if "title" in df.columns:
      df['title'] = df['title'].str.replace('"', "'")
      df=df[['title']]
      duplicate_titles = find_duplicate_titles(df)
      total_duplicates += len(duplicate_titles)
      total_titles += len(df)
print("Total number of duplicate titles across all files:", total_duplicates)
print("Proportion of total duplicate titles across all files:", (total_duplicates/total_titles)*100.00,"%")


<ipython-input-12-98688e85713e>:14: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename))
<ipython-input-12-98688e85713e>:14: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename))


Total number of duplicate titles across all files: 261632
Proportion of total duplicate titles across all files: 2.621276834494028 %
